In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/piano-musics-abc-notation/piano-musics-abc-notation.txt


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
import random

file_path = '/kaggle/input/piano-musics-abc-notation/piano-musics-abc-notation.txt'
with open(file_path, 'r') as file:
    abc_data = file.read()

2024-04-18 10:00:31.237708: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 10:00:31.237814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 10:00:31.413431: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# abc_data

In [4]:
tokens = sorted(set(abc_data))
num_tokens = len(tokens)
token_to_index = {token: i for i, token in enumerate(tokens)}
index_to_token = {i: token for token, i in token_to_index.items()}

In [5]:
# abc_data = abc_data[0: 20]

In [6]:
# abc_data

In [7]:
SEQ_LENGTH = 100
STEP = 1
sequences = []
next_tokens = []

for i in range(0, len(abc_data) - SEQ_LENGTH, STEP):
    sequences.append([token_to_index[token] for token in abc_data[i:i + SEQ_LENGTH]])
    next_tokens.append(token_to_index[abc_data[i + SEQ_LENGTH]])

In [8]:
len(sequences)

1147236

In [9]:
len(next_tokens)

1147236

In [10]:
# # Vectorize the sequences
# X = np.zeros((len(sequences), SEQ_LENGTH, num_tokens), dtype=bool)
# y = np.zeros((len(sequences), num_tokens), dtype=bool)
# for i, sequence in enumerate(sequences):
#     for t, token in enumerate(sequence):
#         X[i, t, token_to_index[token]] = 1
#     y[i, token_to_index[next_tokens[i]]] = 1

X = np.array(sequences)
y = np.array(next_tokens)

In [11]:
X.shape

(1147236, 100)

In [12]:
y.shape

(1147236,)

In [13]:
num_tokens

86

In [14]:
unique_values, counts = np.unique(y, return_counts=True)
print("Unique Values:", unique_values)
print("Counts:", counts)

Unique Values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85]
Counts: [  8886 200009     65  25772    136    729  35675  29311      4     84
   1597   2930  10144  24231   3375   3711  62674  15441   5026   1324
   1810   3346   1148   1017   6216   9262   4175  29004      1  65002
  55861   2775  19911  18980  25496  49738    607     10      6     13
     78     11      9     84      6    191   1171      1      4      1
      3     12    468   3115   1061   4075   4241  26759  10563  50470
  66645  49177  40392  33796   2964   4358    893    162   4495   2115
   1130    600    586   2317    810    938     64     18    539     71
     37   2617   4028  95360   4026   1273]


In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
encoder = OneHotEncoder(categories=[range(num_tokens)], sparse_output=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

In [17]:
y_encoded.shape

(1147236, 86)

In [18]:
num_tokens

86

In [19]:
import numpy as np
from keras.models import Sequential
from keras.layers import BatchNormalization, Flatten, LSTM, Dense, Embedding, Dropout, TimeDistributed, Activation
import random

model = Sequential()
model.add(Embedding(input_dim=86, output_dim=100))

for i in range(3):
    model.add(LSTM(256, return_sequences=True))
    model.add(BatchNormalization())
#     model.add(Dropout(0.2))
    
    
model.add(Dense(num_tokens)) 
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(num_tokens))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model

In [20]:
model.fit(X, y_encoded, batch_size=128, epochs=20)

Epoch 1/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 498s 55ms/step - loss: 1.9848
Epoch 2/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 496s 55ms/step - loss: 1.0313
Epoch 3/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 495s 55ms/step - loss: 0.8526
Epoch 4/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 495s 55ms/step - loss: 0.7156
Epoch 5/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 494s 55ms/step - loss: 0.5988
Epoch 6/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 495s 55ms/step - loss: 0.5045
Epoch 7/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 496s 55ms/step - loss: 0.4323
Epoch 8/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 497s 55ms/step - loss: 0.3773
Epoch 9/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 499s 56ms/step - loss: 0.3296
Epoch 10/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 500s 56ms/step - loss: 0.2947
Epoch 11/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 500s 56ms/step - loss: 0.2695
Epoch 12/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 500s 56ms/step - loss: 0.2456
Epoch 13/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 500s 56ms/step - loss: 0.2292
Epoch 14/20
8963/8963 ━━━━━━━━━━━━━━━━━━━━ 500s 56ms/step - 

In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 100)       │         8,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100, 256)       │       365,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 100, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100, 256)       │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100, 256)       │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100, 86)        │        22,102 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 100, 86)        │           344 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 86)             │       739,686 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 86)             │           344 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 86)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,567,262 (25.05 MB)

 Trainable params: 2,188,460 (8.35 MB)

 Non-trainable params: 1,880 (7.34 KB)

 Optimizer params: 4,376,922 (16.70 MB)

In [22]:
X.shape

(1147236, 100)

In [23]:
y_encoded.shape

(1147236, 86)

In [24]:
def make_array_multiple_of(arr, num):
    remainder = len(arr) % num
    if remainder != 0:
        return arr[:-remainder]
    else:
        return arr

In [25]:
num = SEQ_LENGTH
num

100

In [26]:
def generate_music(seed_sequence, length=100):
    
    generated = seed_sequence
    
    seed_sequence_numeric = []
    
#     print(seed_sequence)
    
    for token in seed_sequence:
        seed_sequence_numeric.append(token_to_index[token])
#         print(token)
        
#     print(seed_sequence_numeric)
        
    seed_sequence_numeric = np.array(seed_sequence_numeric)
    
    seed_sequence_numeric = make_array_multiple_of(seed_sequence_numeric, num)
    
    print(seed_sequence_numeric.shape)
    
    for _ in range(length):
        
        input_seq = seed_sequence_numeric[_ : _+num]
        input_seq = input_seq.reshape(-1, num)
        
        preds = model.predict(input_seq, verbose=0)
        predicted_classes = np.argmax(preds, axis=1)
        next_token = index_to_token[predicted_classes[0]]
        generated += next_token
        
        seed_sequence_numeric = seed_sequence_numeric.reshape(-1)
#         print(seed_sequence_numeric)
#         print(predicted_classes)
        seed_sequence_numeric = np.append(seed_sequence_numeric, predicted_classes)
#         print(seed_sequence_numeric)
        
#     print(generated)

    return generated

In [27]:
abc_data[37:474]

'm"BcB "E7"B2f|"A"ecc c2f|"A"ecc c2c/2d/2|"D"efe "E7"dcB| [1"A"Ace a2:|[2"A"Ace ag=g||\\"D"f2f Fdd|"D"AFA f2e/2f/2|"G"g2g ecd|"Em"efd "A7"cBA|"D"f^ef dcd|"D"AFA f=ef|"G"gfg "A7"ABc |1"D"d3 d2e:|2"D"d3 d2||\n\n"G"g2g B^AB|d2d G3|"Em"GAB "Am"A2A|"D7"ABc "G"BAG|"G"g2g B^AB|d2d G2G|"Em"GAB "Am"A2G|"D7"FGA "G"G3::"D7"A^GA DFA|"G"B^AB G3|"A7"^c=c^c Ace|"D7"fef def|"G"g2g de=f|"E7"e2e Bcd|"Am"c2c "D7"Adc| [1"G"B2A G3:|[2"G"B2A G2F||"Em"E2E G2G|'

In [28]:
seed_sequence = abc_data[37:374]
# seed_sequence
y = generate_music(seed_sequence, length=500)
y[-500:-1]
# print(len(y))

# preds = model.predict(y[0], verbose=0)
# preds

(300,)


'cFA d2e/2f/2|"G"g2g "cc|"Em"ecd "A7"cBA|"D"fgef deB|"D"AFF f2ed|"G"gfg "A7"aBc||1"E"dc d2::|2"D"d2 "2||\n\n"D"g2g g^AB|G2d G3|"Gm"GAB "Am"ABA|"D7"FBc "G"dAG|"A"g2g d2AB|"2d d2B|"Gm"BAB "A7"c2B|"D7"FGA "G"EB:g:|2^/2|"G"gbg "D"B"bm"Bde "A7"e3c|"D7dea 2def|"E"cFA "2:||[G"Bfg "B7"aec|"\n"Emdcd"ec|F[|A"dedfG"":\n"D"f2f "fff|"2B d2|"G""d2B "Gm"c2c|"D7"cB3 "G"deB|"GmA2g "^B||"Cm"c2d "E""dAG dD7"dBe|"D7"AGA "G"G2c:""""gaa"G"d2g gC7aAp#""cB||Gm"ad||"Gm""f ""Bd|"Gmg3E "D""2:13B/B aC7"agf|"E""g"BB "aB:AB"3A7|'

In [29]:
# seq = np.array([[1,2], [2,3]])

# encoder = OneHotEncoder(categories=[range(num_tokens)], )
# en = encoder.fit_transform(seq.reshape(-1, 1))

In [30]:
# seed_sequence = 'f|"A"|'
# y = generate_music(seed_sequence, length=100)

# # print(len(y))

# # preds = model.predict(y[0], verbose=0)
# # preds

In [31]:
# y_test = np.array([[62, 83], [83, 3]])
# y_test

In [32]:
# encoder = OneHotEncoder(categories=[range(num_tokens)], sparse=False)
# y_encoded = encoder.fit_transform(y_test.reshape(-1, 1))
# y_encoded

In [33]:
# y_encoded[0].shape

In [34]:
# preds = model.predict(y_encoded, verbose=0)
# preds

In [35]:
# predicted_classes = np.argmax(preds, axis=1)
# predicted_classes

In [36]:
# seed_sequence = 'f|"A"ecc c2f|"A"ecc c2f|"A"ecc c2f|'
# generated_music = generate_music(seed_sequence, length=500)
# print(generated_music)